In [1]:
!pip install -q opendatasets

from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [2]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

inference_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
train_dataset = datasets.ImageFolder(root='/Users/luka/PyCharmProjects/simple-defect-detection/data/casting_data/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='/Users/luka/PyCharmProjects/simple-defect-detection/data/casting_data/train', transform=inference_transforms) #TODO: actually create a val dataset

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
device

device(type='mps')

In [5]:
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer to match the number of classes in your new dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)

model = model.to(device)

In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

In [7]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Adjust labels to be a float tensor (required for BCEWithLogitsLoss)
        labels = labels.float()

        optimizer.zero_grad()

        outputs = model(inputs).squeeze()  # Remove unnecessary dimensions
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = torch.sigmoid(outputs) > 0.5  # Applying threshold at 0.5
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {train_accuracy}%")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.float()

            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predicted = torch.sigmoid(outputs) > 0.5
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {val_accuracy}%")

print("Training complete")

Epoch 1/1, Loss: 0.25763396732509136, Accuracy: 88.42152872003618%
Validation Loss: 2.18322100609667, Accuracy: 56.83702698628072%
Training complete


In [8]:
def predict(img_path):
    image = Image.open(img_path)

    transformed_image = inference_transforms(image)
    transformed_image = transformed_image.unsqueeze(0)  # Add batch dimension
    transformed_image = transformed_image.to(device)

    model.eval()

    with torch.no_grad():
        outputs = model(transformed_image)
        # Convert output logits to probability using sigmoid
        probability = torch.sigmoid(outputs).item()
        predicted_class = 'defective' if probability > 0.5 else 'non-defective'

    print(f'Predicted class: {predicted_class}, Probability: {probability}')

In [13]:
predict('/Users/luka/PyCharmProjects/simple-defect-detection/data/casting_data/test/ok_front/cast_ok_0_239.jpeg')

Predicted class: non-defective, Probability: 0.013920432887971401
